# Python Review Project
## Part 2 - Applying ML to Cancer Data Analysis
For this project, you will create a machine learning model to predict the stage of cancer (I, II, III, or IV) from both RNA and protein-level gene expression for clear cell renal cell carcinoma (CCRCC) in CPTAC. Stage of cancer can be found using the tumor_stage_pathological column within the CPTAC clinical data. You can access the data the exact same way as BRCA, substituting the accession code.

**1)** Select what features to include in the model by finding the top 5 most differentially express proteins between Stage I and Stage III patients in CPTAC protein data. Repeat this process to find the top 5 most differential expression RNA between Stage I and Stage III patients in the CPTAC RNA data. Use tumor_stage_pathological in the CPTAC clinical data.

In [132]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [1]:
# download the corresponding cancer data set
import cptac

cptac.download(dataset="Ccrcc")
ccrcc = cptac.Ccrcc()

cptac warning: Your version of cptac (1.1.2) is out-of-date. Latest is 1.5.13. Please run 'pip install --upgrade cptac' to update it. (C:\Users\Owner\anaconda3_new\Lib\threading.py, line 975)


In [64]:
ccrcc.list_data()

Below are the dataframes contained in this dataset and their dimensions:

clinical
	194 rows
	171 columns
CNV
	110 rows
	19285 columns
followup
	352 rows
	27 columns
medical_history
	370 rows
	4 columns
methylation
	107 rows
	15885 columns
phosphoproteomics
	194 rows
	81550 columns
phosphoproteomics_gene
	194 rows
	6127 columns
proteomics
	194 rows
	11710 columns
somatic_mutation
	8350 rows
	3 columns
transcriptomics
	185 rows
	19275 columns


In [26]:
# get dataframes for protein, RNA, and clinical data
protein_data = ccrcc.get_proteomics()
protein_data.columns = protein_data.columns.get_level_values(0)
rna_data = ccrcc.get_transcriptomics()
clinical_data = ccrcc.get_clinical()

In [36]:
print(protein_data.shape)
print(rna_data.shape)
print(clinical_data.shape)

(194, 11710)
(185, 19275)
(194, 171)


In [146]:
# only analyze patients shared between data frames
clinical_trimmed = clinical_data.loc[np.intersect1d(rna_data.index, clinical_data.index)]
protein_trimmed = protein_data.loc[np.intersect1d(rna_data.index, protein_data.index)]

# log scale rna_data
rna_data = np.log2(rna_data)

# drop columns with NA values in protein_data and rna_data
protein_trimmed = protein_trimmed.dropna(axis='columns')
rna_data = rna_data.dropna(axis='columns')

In [88]:
# find top 5 most differentially expressed proteins between stage i and stage iii
stage_i_mask = (clinical_trimmed.loc[:, 'tumor_stage_pathological'] == "Stage I")
stage_iii_mask = (clinical_trimmed.loc[:, 'tumor_stage_pathological'] == "Stage III")

stage_i_ptn = protein_trimmed.loc[stage_i_mask, :]
stage_iii_ptn = protein_trimmed.loc[stage_iii_mask, :]

# compute mean expression of every protein within each group
stage_i_ptn_means = stage_i_ptn.mean(axis = 0, skipna = True)
stage_iii_ptn_means = stage_iii_ptn.mean(axis = 0, skipna = True)

# compute difference in mean protein expression between groups
diff_mean_ptn_exp = stage_i_ptn_means - stage_iii_ptn_means
diff_mean_ptn_exp = np.asarray(diff_mean_ptn_exp)

# converting NA values to 0 in order to compute maximum
np.nan_to_num(diff_mean_ptn_exp, copy=False)

# find top 5 most differentially expressed proteins
ptn_col_indices = []
for i in range(5):
    max_index = diff_mean_ptn_exp.argmax()
    ptn_col_indices.append(max_index)
    diff_mean_ptn_exp[max_index] = 0
    
diffeq_ptns = []
for index in ptn_col_indices:
    ptn_name = protein_trimmed.columns[index]
    diffeq_ptns.append(ptn_name)

In [148]:
# find top 5 most differentially expressed RNA between stage i and stage iii
stage_i_rna = rna_data.loc[stage_i_mask, :]
stage_iii_rna = rna_data.loc[stage_iii_mask, :]

# compute mean expression of every protein within each group
stage_i_rna_means = stage_i_rna.mean(axis = 0, skipna = True)
stage_iii_rna_means = stage_iii_rna.mean(axis = 0, skipna = True)

# compute difference in mean protein expression between groups
diff_mean_rna_exp = stage_i_rna_means - stage_iii_rna_means
diff_mean_rna_exp = np.asarray(diff_mean_rna_exp)

# converting NA values to 0 in order to compute maximum
np.nan_to_num(diff_mean_rna_exp, copy=False)

# find top 5 most differentially expressed proteins
rna_col_indices = []
for i in range(5):
    max_index = diff_mean_rna_exp.argmax()
    rna_col_indices.append(max_index)
    diff_mean_rna_exp[max_index] = 0
    
diffeq_rnas = []
for index in ptn_col_indices:
    rna_name = rna_data.columns[index]
    diffeq_rnas.append(rna_name)

In [149]:
print(diffeq_ptns)
print(diffeq_rnas)

['FTL', 'HBZ', 'HBA2', 'CMA1', 'HBB']
['EVI2B', 'GAPDH', 'GALNT2', 'CENPB', 'GALNT4']


**2)** Create a new dataframe of your select features, where the rows are the patients and the columns are the expression values of genes you selected in step 1 (X data).

In [150]:
# subsetting protein columns
X_features = protein_trimmed[['FTL', 'HBZ', 'HBA2', 'CMA1', 'HBB']]

# subsetting rna columns
X_features.insert(len(X_features.columns), 'EVI2B', rna_data.loc[:, 'EVI2B'])
X_features.insert(len(X_features.columns), 'GAPDH', rna_data.loc[:, 'GAPDH'])
X_features.insert(len(X_features.columns), 'GALNT2', rna_data.loc[:, 'GALNT2'])
X_features.insert(len(X_features.columns), 'CENPB', rna_data.loc[:, 'CENPB'])
X_features.insert(len(X_features.columns), 'GALNT4', rna_data.loc[:, 'GALNT4'])

**3)** Create a separate list of the patients' cancer stages, i.e. tumor_stage_pathological (y data).

In [151]:
tumor_stages = clinical_trimmed.loc[:, "tumor_stage_pathological"]

**4)** Scale and encode your features and target.

In [152]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

scaler = StandardScaler()
encoder = LabelEncoder()

scaled_data = scaler.fit_transform(X_features)
encoded_stages = encoder.fit_transform(tumor_stages)

**5)** Create a train test split of your X and y data with train_size=0.7.

In [153]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_features, encoded_stages, train_size=0.7)

**6)** Write code to test the accuracy of all 4 classification models we covered in this class (i.e. KNeighborsClassifier, DecisionTreeClassifier, and MLPClassifier, GaussianNB). Since the accuracy of the models will change depending on the train-test split, you will need to run each model 10 times and find the average accuracy between all runs.

In [154]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB

In [155]:
# run each model 10 times and find average accuracy between all runs
classifiers = [KNeighborsClassifier(),
              DecisionTreeClassifier(),
              MLPClassifier(),
              GaussianNB()]
classifiers_names = ['Nearest Neighbors',
                    'Decision Tree',
                    'Multilayer Perceptron',
                    'Naive-Bayes']
classifiers_accuracy = []

# iterate through list of classification methods
for i in range(4):
    classifier = classifiers[i]
    accuracies = []
    
    # run each model 10 times
    for i in range(10):
        classifier.fit(X_train, y_train)
        y_pred = classifier.predict(X_test)
        
        accuracy = sum(y_pred == y_test) / len(y_test)
        accuracies.append(accuracy * 100)
        
    classifiers_accuracy.append(np.mean(accuracies))

**7)** Compare the 4 mean accuracies and identify which model is best.

In [156]:
print('After 10 simulations, the average accuracy for each classification model is as follows:')
for i in range(4):
    print(f'{classifiers_names[i]}: {classifiers_accuracy[i]}%')

After 10 simulations, the average accuracy for each classification model is as follows:
Nearest Neighbors: 67.85714285714286%
Decision Tree: 61.25%
Multilayer Perceptron: 69.28571428571428%
Naive-Bayes: 69.64285714285714%


Comparing the mean accuracies between the 4 classification models, the Naive-Bayes classification model is best for this data. The Multilayer Perceptron was a close second-best choice of model for this data.